<a href="https://colab.research.google.com/github/mrtkp9993/QuantitaveFinanceExamplesPy/blob/main/04_volatility_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [10]:
kozaa = yf.download("KOZAA.IS", period = "max", start = "2006-01-03")

[*********************100%***********************]  1 of 1 completed


In [11]:
kozaa.drop(['Close'], axis=1, inplace=True)
kozaa.rename(columns={'Adj Close':'Close'}, inplace=True)
kozaa

,Open,High,Low,Close,Volume
Date,,,,,
2006-01-03,0.680429,0.697951,0.674588,0.677509,9802944
2006-01-04,0.677509,0.697951,0.654146,0.680429,23936953
2006-01-05,0.674588,0.686270,0.671668,0.677509,10254562
2006-01-06,0.695030,0.706712,0.686270,0.703791,21481786
2006-01-09,0.703791,0.706712,0.686270,0.703791,21481786
...,...,...,...,...,...
2021-08-23,15.650000,15.720000,15.350000,15.650000,24798852
2021-08-24,15.700000,15.900000,15.500000,15.510000,16496741
2021-08-25,15.500000,15.640000,15.130000,15.480000,21069675


In [15]:
fig = go.Figure(
    data=[
          go.Candlestick(
            x = kozaa.index,
            close = kozaa['Close'],
            high = kozaa['High'],
            low = kozaa['Low'],
            open = kozaa['Open']
          )
          ]
          )

fig.show()

In [21]:
# Volatility estimation
# Sources:
# D. Yang and Q. Zhang. Drift-independent volatility estimation based on high, low, open and close prices
# L.C.G. Rogers and S.E. Satchell. Estimating variance from high, low and closing prices. Annals of Applied Probability, 1:504–512, 1991.
kozaa['o'] = np.log(kozaa.Open) - np.log(kozaa.Close.shift(1))
kozaa['u'] = np.log(kozaa.High) - np.log(kozaa.Open)
kozaa['d'] = np.log(kozaa.Low) - np.log(kozaa.Open)
kozaa['c'] = np.log(kozaa.Close) - np.log(kozaa.Open)

kozaa.dropna(inplace=True)

In [43]:
n = kozaa.shape[0]

# Classical estimator
Vcc = (1 / (n - 1)) * np.sum(
    (kozaa['o'] + 
     kozaa['c'] - 
     np.mean(kozaa['o']) - 
     np.mean(kozaa['c']))**2  
)

# Rogers - Satchell estimator
Vrs = (1 / n) * np.sum((kozaa['u'] * (kozaa['u'] - kozaa['c'])) + 
                       (kozaa['d'] * (kozaa['d'] - kozaa['c'])))

# Yang - Zang estimator
k = 0.34 / (1.34 + (n+1)*(n-1)) 
Vo = (1 / (n-1)) * np.sum((kozaa['o'] - np.mean(kozaa['o']))**2)
Vc = (1 / (n-1)) * np.sum((kozaa['c'] - np.mean(kozaa['c']))**2)

Vyz = Vo + k * Vc + (1-k) * Vrs

In [45]:
print(np.std(kozaa.Close.diff()))
print(Vcc)
print(Vrs)
print(Vyz)

0.1709884606608804
0.0013517026486314192
0.000976641474364263
0.0016533331705707163
